# SVG(inf) for LQG

## Introduction
In this notebook we implement a simplified version of SVG($\infty$) for the LQG problem. Our intention is to show how one can learn a parameterized policy in LQG via Stochastic Value Gradients. Note that even if we successfully learn a policy, we do not necessarily know what components of the SVG framework were instrumental in doing so. The focus of the rest of our work will be on analyzing the tenets of this framework based on **gradient estimation quality** and **performance curvature approximation**.

### Imports

In [1]:
from __future__ import annotations
import itertools
from datetime import date
from typing import Optional

import pytorch_lightning as pl
import torch
import torch.nn as nn
import wandb
from ray.rllib import RolloutWorker
from ray.rllib import SampleBatch
from ray.rllib.evaluation.metrics import collect_episodes, summarize_episodes
from raylab.policy.modules.actor import DeterministicPolicy
from torch.optim import Optimizer
from torch.utils.data import ConcatDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch.utils.data import TensorDataset
from tqdm.auto import trange

import lqsvg.torch.named as nt
from lqsvg.envs import lqr
from lqsvg.envs.lqr.gym import TorchLQGMixin
from lqsvg.policy.time_varying_linear import LQGPolicy

from data import TrajectoryData  # pylint:disable=wrong-import-order
from models import MonteCarloSVG  # pylint:disable=wrong-import-order
from models import glorot_init_model  # pylint:disable=wrong-import-order
from policy import make_worker  # pylint:disable=wrong-import-order
from tqdm_util import collect_with_progress  # pylint:disable=wrong-import-order
from utils import group_batch_episodes  # pylint:disable=wrong-import-order
from utils import suppress_dataloader_warning  # pylint:disable=wrong-import-order

---
## Algorithm

In [2]:
def create_dataset(rollout_worker: RolloutWorker, hparams: dict) -> pl.LightningDataModule:
    del rollout_worker
    return DataModule(hparams)

In [3]:
def create_optimizer(policy: DeterministicPolicy, hparams: dict) -> Optimizer:
    return torch.optim.Adam(policy.parameters(), lr=hparams["policy_lr"])

In [4]:
def collect_trajs(rollout_worker: RolloutWorker, dataset: pl.LightningDataModule, hparams: dict):
    dataset.collect_trajectories(rollout_worker, n_trajs=hparams["trajs_per_iter"])

In [5]:
def optimize_model(model: pl.LightningModule, dataset: pl.LightningDataModule, hparams: dict):
    early_stopping = pl.callbacks.EarlyStopping(
        model.early_stop_on,
        min_delta=hparams["improvement_delta"],
        patience=hparams["patience"],
        mode="min",
        strict=True,
    )
    trainer = pl.Trainer(callbacks=[early_stopping], max_epochs=1000, progress_bar_refresh_rate=0)
    with suppress_dataloader_warning():
        trainer.fit(model, datamodule=dataset)

    # Logging
    wandb.log({"model_epochs": trainer.current_epoch + 1}, commit=False)
    print("Model epochs:", trainer.current_epoch + 1)

In [6]:
def step_policy(policy: DeterministicPolicy, model: pl.LightningModule, optimizer: Optimizer, hparams: dict):
    svg = MonteCarloSVG(policy, model.model)
    optimizer.zero_grad()
    loss = -svg.value(hparams["svg_samples"])
    loss.backward()
    optimizer.step()

In [7]:
def svg_inf(policy: DeterministicPolicy, model: pl.LightningModule, rollout_worker: RolloutWorker, hparams: dict):    
    artifact = wandb.Artifact(f"svg(inf)-lqg({model.model.n_state}, {model.model.n_ctrl}, {model.model.horizon})", type="model")
    collect_metrics = CollectMetrics()

    ### Main algorithm logic ###
    dataset = create_dataset(rollout_worker, hparams)
    optimizer = create_optimizer(policy, hparams)
    for itr in trange(hparams["iterations"], desc="SVG(inf) iteration"):
        collect_trajs(rollout_worker, dataset, hparams)
        optimize_model(model, dataset, hparams)
        step_policy(policy, model, optimizer, hparams)
    ############################
        
        # Logging
        wandb.log({"svg_iter": iter}, commit=False)
        wandb.log(collect_metrics(rollout_worker))
        
        if itr % 10 == 0:
            

### Logging

In [8]:
class CollectMetrics:
    def __init__(self):
        self.episode_history = []
        self.to_be_collected = []

    def __call__(self, worker: RolloutWorker, min_history: int = 100, timeout_seconds: int = 180) -> dict:
        # Collect worker metrics.
        episodes, self.to_be_collected = collect_episodes(
            worker, to_be_collected=self.to_be_collected, timeout_seconds=timeout_seconds
        )
        orig_episodes = list(episodes)
        missing = min_history - len(episodes)
        if missing > 0:
            episodes.extend(self.episode_history[-missing:])
            assert len(episodes) <= min_history
        self.episode_history.extend(orig_episodes)
        self.episode_history = self.episode_history[-min_history:]
        return summarize_episodes(episodes, orig_episodes)

### Modules

In [9]:
class DataModule(pl.LightningDataModule):
    def __init__(self, hparams: dict):
        super().__init__()
        
        self.batch_size: float = hparams["dataset_batch_size"]
        self.train_val_split: tuple[float, float] = hparams["train_val_split"]

        self.full_dataset = None
        self.train_dataset, self.val_dataset = None, None
        self._itr_datasets: list[TensorDataset] = []
        
    def collect_trajectories(self, rollout_worker: RolloutWorker, n_trajs: int):
        worker = rollout_worker
        self._check_rollout_worker(worker)
        
        sample_batch = collect_with_progress(worker, n_trajs)
        sample_batch = group_batch_episodes(sample_batch)
        trajs = sample_batch.split_by_episode()
        self._check_collected_trajs(trajs, worker, n_trajs)
        
        traj_dataset = TrajectoryData.trajectory_dataset(trajs)
        self._itr_datasets += [traj_dataset]
        self.full_dataset = ConcatDataset(self._itr_datasets)
        
    def setup(self, stage: Optional[str] = None):
        del stage
        train_frac, _ = self.train_val_split
        train_trajs = int(train_frac * len(self.full_dataset))
        val_trajs = len(self.full_dataset) - train_trajs

        self.train_dataset, self.val_dataset = random_split(self.full_dataset, (train_trajs, val_trajs))

    def train_dataloader(self) -> DataLoader:
        # pylint:disable=arguments-differ
        return DataLoader(self.train_dataset, shuffle=True, batch_size=self.batch_size)

    def val_dataloader(self) -> DataLoader:
        # pylint:disable=arguments-differ
        return DataLoader(self.val_dataset, shuffle=False, batch_size=self.batch_size)

    @staticmethod
    def _check_rollout_worker(worker: RolloutWorker):
        assert worker.rollout_fragment_length == worker.env.horizon * worker.num_envs
        assert worker.batch_mode == "truncate_episodes"
        
    @staticmethod
    def _check_collected_trajs(trajs: list[SampleBatch], worker: RolloutWorker, n_trajs: int):
        traj_counts = [t.count for t in trajs]
        assert all(c == worker.env.horizon for c in traj_counts), traj_counts
        total_ts = sum(t.count for t in trajs)
        assert total_ts == n_trajs * worker.env.horizon, total_ts        

In [10]:
class EnvModel(pl.LightningModule):
    early_stop_on: str = "val/loss"

    def __init__(self, policy: LQGPolicy, hparams: dict):
        super().__init__()
        self.model = policy.module.model

        self.hparams.learning_rate = hparams["model_lr"]
        glorot_init_model(self.model)

    def configure_optimizers(self):
        params = nn.ParameterList(
            itertools.chain(self.model.trans.parameters(), self.model.init.parameters())
        )
        optim = torch.optim.Adam(params, lr=self.hparams.learning_rate)
        return optim

    def forward(self, obs: Tensor, act: Tensor, new_obs: Tensor) -> Tensor:
        """Batched trajectory log prob."""
        # pylint:disable=arguments-differ
        return self.model.log_prob(obs, act, new_obs)

    def _compute_loss_on_batch(
        self, batch: tuple[Tensor, Tensor, Tensor], batch_idx: int
    ) -> Tensor:
        del batch_idx
        obs, act, new_obs = (x.refine_names("B", "H", "R") for x in batch)
        return -self(obs, act, new_obs).mean()

    def training_step(
        self, batch: tuple[Tensor, Tensor, Tensor], batch_idx: int
    ) -> Tensor:
        # pylint:disable=arguments-differ
        loss = self._compute_loss_on_batch(batch, batch_idx)
        self.log("train/loss", loss)
        return loss

    def validation_step(
        self, batch: tuple[Tensor, Tensor, Tensor], batch_idx: int
    ) -> Tensor:
        # pylint:disable=arguments-differ
        loss = self._compute_loss_on_batch(batch, batch_idx)
        self.log("val/loss", loss)
        return loss

---
## Experiments

In [11]:
def calver() -> str:
    today = date.today()
    return f"{today.month}.{today.day}.0"

print("CalVer:", calver())

CalVer: 3.16.0


In [ ]:
run = wandb.init(
    name="SVG(inf)",
    project="LQG-SVG",
    entity="angelovtt",
    tags=[calver()],
    mode="offline",
    save_code=True,
    job_type="train",
)

with run, nt.suppress_named_tensor_warning():
    env_config = dict(n_state=2, n_ctrl=2, horizon=100, num_envs=100)
    hparams = {
        "iterations": 500,
        "trajs_per_iter": 2000,
        "policy_lr": 3e-4,
        "improvement_delta": 0.0,
        "patience": 3,
        "svg_samples": 32,
        "dataset_batch_size": 64,
        "train_val_split": (0.8, 0.2),
        "model_lr": 1e-3,
        "env_config": env_config,
    }
    
    run.config.update(hparams)
    worker = make_worker(env_config)
    rllib_policy = worker.get_policy()
    policy = rllib_policy.module.actor
    model = EnvModel(rllib_policy, run.config)
    
    svg_inf(policy, model, worker, run.config)

wandb: W&B syncing is set to `offline` in this directory.  Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.


SVG(inf) iteration:   0%|          | 0/500 [00:00<?, ?it/s]

Collecting:   0%|          | 0/2000 [00:00<?, ?traj/s]

2021-03-16 08:52:57,674	WARNING deprecation.py:33 -- DeprecationWarning: `env_index` has been deprecated. Use `episode.env_id` instead. This will raise an error in the future!
GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name  | Type      | Params
------------------------------------
0 | model | LQGModule | 3.6 K 
------------------------------------
3.6 K     Trainable params
0         Non-trainable params
3.6 K     Total params


Model epochs: 427


...

Model epochs: 5


Collecting:   0%|          | 0/2000 [00:00<?, ?traj/s]

GPU available: False, used: False
TPU available: None, using: 0 TPU cores

  | Name  | Type      | Params
------------------------------------
0 | model | LQGModule | 3.6 K 
------------------------------------
3.6 K     Trainable params
0         Non-trainable params
3.6 K     Total params


Model epochs: 7


Collecting:   0%|          | 0/2000 [00:00<?, ?traj/s]